In [1]:
from pyLAMMPS import LAMMPS_setup

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

********* JAX NOT FOUND *********
 PyMBAR can run faster with JAX  
 But will work fine without it   
Either install with pip or conda:
      pip install pybar[jax]     
               OR                
      conda install pymbar       
*********************************


# pyLAMMPS

This module enables users to perform molecular dynamics simulations utilizing LAMMPS with the any force field provided as toml/json file. 
There is the possiblity do provide data and LAMMPS compatible parameter files, or to build a system and write all necessary input using pyLAMMPS, PLAYMOL, and moleculegraph.

1) Read in the YAML files to define the system and simulation/sampling settings.

In [2]:
lammps_setup = LAMMPS_setup( system_setup = "input/setup.yaml", 
                             simulation_default = "input/defaults.yaml",
                             simulation_ensemble = "input/ensemble.yaml",
                             simulation_sampling = "input/sampling.yaml",
                             submission_command = "qsub"
                            )

## Setting up a simulation pipeline

In this section the possibility to setup a simulation folder, along with a simulation pipeline using several ensembles, is provided.

1) Setup simulation and build initial system (if not provided)


In [3]:
# Define the simulation folder
simulation_folder = "md_thermo"

# Define the ensembles that should be simulated (definition what each ensemble means is provided in yaml file)
ensembles = [ "em", "npt" ] 

# Define the simulation time per ensemble in nano seconds (for em the number of iterations is provided in the ensemble yaml)
simulation_times = [ 0, 0.1 ]

# Define initial systems, in case the simulation should be continued from a prior simulation.
# In that case, provide one initial structure for each temperature & pressure state.
# If the simulation should start from an initial configuration, provide an empty list.
initial_systems = [ ]

# Define if there is already a force field file
ff_file = ""

# Provide kwargs that should be passed into the input template directly
input_kwargs = {  }

# Define number of copies
copies = 2

# Define if the inital system should build locally or with the cluster
on_cluster = False

# Define the starting number for the first ensemble ( 0{off_set}_ensemble )
off_set    = 0

lammps_setup.prepare_simulation( folder_name = simulation_folder, ensembles = ensembles, simulation_times = simulation_times,
                                 initial_systems = initial_systems, input_kwargs = input_kwargs, copies = copies,
                                 ff_file = ff_file, on_cluster = on_cluster,  off_set = off_set )

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2,) + inhomogeneous part.

In [5]:
from pyLAMMPS.tools import LAMMPS_molecules

In [7]:
system_molecules = [ mol for mol in lammps_setup.system_setup["molecules"] if mol["number"] > 0 ]

lmp_mol = LAMMPS_molecules( mol_str = [ mol["graph"] for mol in system_molecules ],
                                                 force_field_paths = lammps_setup.system_setup["paths"]["force_field_paths"] 
                                                ) 

2) Submit jobs to cluster

In [4]:
# Submit the simulations
lammps_setup.submit_simulation()


Submitting simulations at Temperature = 343 K, Pressure = 4 bar

Submitting job: /home/st/st_st/st_ac137577/workspace/software/pyLAMMPS/example/butane_hexane/md_thermo/temp_343_pres_4/copy_0/job_343_4.sh
11356787


Submitting job: /home/st/st_st/st_ac137577/workspace/software/pyLAMMPS/example/butane_hexane/md_thermo/temp_343_pres_4/copy_1/job_343_4.sh
11356788


Submitting job: /home/st/st_st/st_ac137577/workspace/software/pyLAMMPS/example/butane_hexane/md_thermo/temp_343_pres_4/copy_2/job_343_4.sh
11356789




## Extract sampled properties

In [3]:
# Extract properties from LAMMPS and analyse them

# Define analysis folder
analysis_folder = "md_thermo"

# Define analysis ensemble
ensemble = "01_npt"  

# Properties to extract
properties = ["temperature", "potential energy", "kinetic energy", "enthalpy"]

# Suffix of output file
output_suffix = "energy"

# Percentage to discard from beginning of the simulation
fraction = 0.25

lammps_setup.analysis_extract_properties( analysis_folder = analysis_folder, ensemble = ensemble, extracted_properties = properties, 
                                          output_suffix = output_suffix, fraction = fraction )

Temperature: 343, Pressure: 4
   /home/st/st_st/st_ac137577/workspace/software/pyLAMMPS/example/butane_hexane/md_thermo/temp_343_pres_4/copy_0/01_npt/npt.energy
   /home/st/st_st/st_ac137577/workspace/software/pyLAMMPS/example/butane_hexane/md_thermo/temp_343_pres_4/copy_1/01_npt/npt.energy
   /home/st/st_st/st_ac137577/workspace/software/pyLAMMPS/example/butane_hexane/md_thermo/temp_343_pres_4/copy_2/01_npt/npt.energy


Averaged values over all copies:

            property         mean        std      unit
0       temperature   343.166880   0.073278         K
1  potential energy -1650.887479  17.900399  kcal/mol
2    kinetic energy  5113.553718   1.092377  kcal/mol
3          enthalpy  3474.961667  18.004401  kcal/mol 

